In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

## Additional RAM

In [ ]:
# But it is no more proposed get more RAM in a free way
[1]*10**10

## data

In [29]:
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"
ratings = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv"

In [30]:
#Cleaned data base with scrapped books
df_books = pd.read_csv(books, sep = "|")

In [31]:
#ratings of Book Crossing
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

In [32]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [33]:
df_books.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN


In [64]:
df_ratings[df_ratings.ISBN == "000648302X"]

,User-ID,ISBN,Book-Rating
45384,11676,000648302X,8
174443,37950,000648302X,9
373708,89624,000648302X,9
412327,98544,000648302X,0
417691,99328,000648302X,3
496961,119654,000648302X,8
499864,120610,000648302X,7
654880,158589,000648302X,8
659263,160016,000648302X,4
718973,174092,000648302X,0


## Tests merge

In [ ]:
DF_CatUsers = pd.DataFrame( {"UserID": [1,1,1,2,2,3,3,3,3], 
                             "ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN2", "ISBN3", "ISBN5", "ISBN6"], 
                             "Ratings": [4,5,3,2,1,4,5,4,4]})
DF_CatUsers

In [ ]:
DF_CatBooks = pd.DataFrame( {"ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN6", "ISBN7", "ISBN8"], 
                             "features": ["Blabla1", "Blabla2", "Blabla3", "Blabla4", "Blabla5", "Blabla6", "Blabla7", "Blabla8"]})
DF_CatBooks

In [ ]:
pd.merge(DF_CatUsers, DF_CatBooks, on = "ISBN")

## Merge

In [75]:
#Join of ratings and books data base, according to ISBN which is a common column
df_bert = pd.merge(df_ratings, df_books, left_on = "ISBN", right_on = "isbn")  
len(df_bert)

110801

In [76]:
#I will only embeds lines with a not empty book_description
lineOK = df_bert[df_bert["book_description"].isna() == False]
len(lineOK)

75976

In [77]:
#Creation of an additional column in order to put the future book_description embedding
df_bert = df_bert.assign(description_bert = 0)

In [78]:
lineOK.index[997:1002]

Int64Index([1549, 1550, 1551, 1552, 1553], dtype='int64')

In [38]:
len(df_bert)

110801

In [95]:
df_bert.head()

,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,276725,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
1,2313,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
2,6543,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
3,8680,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
4,10314,034545104X,9,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0


In [96]:
len(df_bert)

110801

## Test descriptions

In [ ]:
for d in df_books["book_description"][:3]:
    print(d)

## Bert

### Loading

In [39]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.7MB/s 
     |████████████████████████████████| 901kB 53.1MB/s 
     |████████████████████████████████| 3.3MB 54.2MB/s 


In [40]:
from transformers import BertTokenizer, BertModel

In [41]:
#I will use an already trained BERT network
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Tokenizer and embedding

In [58]:
#Use of pretrained model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [60]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [61]:
#With use of GPU I will save lots of time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [84]:
sent_list = df_bert.book_description[100].split(".")
sent_list = [sent for sent in sent_list if len(sent) > 3]
len(sent_list)

2

In [88]:
"bonjour il n'y a pas de point".split(".")

["bonjour il n'y a pas de point"]

In [89]:
#execution in about 16min

with torch.no_grad():
  for i_line in lineOK.index:
    
    #one book description, potentially composed of several sentences
    sent_list = df_bert.book_description[i_line].split(".")
    #I only keep the not empty results of the previous split
    sent_list = [sent for sent in sent_list if len(sent) > 3]

    #Output of sentences tokenization
    input_ids = []
    attention_masks = []

    #For each sentence in the current book_description
    for sent in sent_list:

      #print(len(sent.split()))
      #Tokenize all of the sentences and map the tokens to thier word IDs.
      encoded_dict = tokenizer.encode_plus(
                                  sent,                      # Sentence to encode.
                                  add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                  max_length = 64,           # Pad & truncate all sentences.
                                  pad_to_max_length = True,
                                  return_attention_mask = True,   # Construct attn. masks.
                                  return_tensors = 'pt',     # Return pytorch tensors.
                        )
              
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
              
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

    if (len(input_ids) > 0):
      # Convert the lists into tensors.
      input_ids_t = torch.cat(input_ids, dim = 0)
      attention_masks_t = torch.cat(attention_masks, dim = 0)

      #use Bert to embedds each tokenized sentences of the current book_description
      outputs = model(input_ids_t.to(device), attention_masks_t.to(device))
      hidden_states = outputs[2]

      # `hidden_states`       has shape [13 layers, len(input_ids), 64, 768] 
      # => hidden_states[-2]  has shape [len(input_ids), 64, 768] 
      token_vecs = hidden_states[-2]
      #print(token_vecs.shape)

      #mean elements by elements of each token (64 tokens in total) of each embedded sentences of the current book_description
      #=> sentence_embedding  has shape [len(input_ids), 768]: 
      sentence_embedding = torch.mean(token_vecs, dim=1)
      #print(sentence_embedding.shape)

      #I only want to keep one embedding of dim 768 for each book_descriptiondescription
      #(and not len(input_ids) x 768)
      sentence_embedding = torch.mean(sentence_embedding, dim=0)
      #print(sentence_embedding.size())

      #I get the values from GPU, and I put it in df_bert.description_bert column
      sentence_embedding = sentence_embedding.cpu().numpy()
      l = len(sentence_embedding)
      df_bert.description_bert[i_line] = [str(sentence_embedding[i]) for i in range(l)]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [90]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_bert.book_description[0])
print('Token IDs:', df_bert.description_bert[0])

Original:  Twenty years after having an affair with legendary artist George Gabriel as a college student, Genny Haviland is accused of drugging and suffocating the painter, despite her claim that she only did it to ease her former lover to a painless ...
Token IDs: ['-0.20497975', '0.032238785', '0.0620426', '-0.38984668', '0.56584084', '0.072482936', '-0.040511448', '0.07431886', '0.5117456', '0.44012055', '0.14020276', '-0.2106059', '0.012214379', '0.14486921', '-0.28126383', '0.5014722', '0.1487891', '0.12371722', '0.12632164', '0.24035808', '0.20319372', '0.10701631', '-0.5470639', '0.25413114', '0.4129166', '0.45108405', '-0.31095874', '-0.058207713', '-0.4434792', '0.5209229', '0.6270331', '-0.070613965', '0.33900243', '-0.25007826', '-0.03216935', '-0.25887477', '-0.114583276', '-0.012186671', '0.11017257', '0.362', '-0.6957621', '-0.040874116', '0.26674026', '-0.15845495', '0.36983103', '-0.2434609', '0.5059105', '-0.30345148', '-0.06425663', '-0.08230286', '-0.777833', '0.2205

In [91]:
df_bert.to_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|", index = False)

# Train / Test data creation

In [5]:
#We get the bert embedding of book_description 
df_bert = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")

In [6]:
#We only use the not empty book_description lines
lineOK = df_bert[df_bert["book_description"].isna() == False]
len(lineOK)

75976

In [92]:
print(len(df_bert))
df_bert.head()

110801


,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,276725,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"[-0.20497975, 0.032238785, 0.0620426, -0.38984..."
1,2313,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"[-0.20497975, 0.032238785, 0.0620426, -0.38984..."
2,6543,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"[-0.20497975, 0.032238785, 0.0620426, -0.38984..."
3,8680,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"[-0.20497975, 0.032238785, 0.0620426, -0.38984..."
4,10314,034545104X,9,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"[-0.20497975, 0.032238785, 0.0620426, -0.38984..."


In [7]:
X_data = df_bert[["User-ID", "description_bert"]]
X_data.head()

,User-ID,description_bert
0,276725,"['-0.20497975', '0.032238785', '0.0620426', '-..."
1,2313,"['-0.20497975', '0.032238785', '0.0620426', '-..."
2,6543,"['-0.20497975', '0.032238785', '0.0620426', '-..."
3,8680,"['-0.20497975', '0.032238785', '0.0620426', '-..."
4,10314,"['-0.20497975', '0.032238785', '0.0620426', '-..."


In [8]:
Y_data = df_bert["Book-Rating"]
Y_data.head()

0    0
1    5
2    0
3    5
4    9
Name: Book-Rating, dtype: int64

In [9]:
len(X_data)

110801

In [9]:
MIN = min(df_bert["User-ID"])
MAX = max(df_bert["User-ID"])
print("min:", MIN)
print("max:", MAX)

min: 8
max: 278854


In [10]:
#Transformation of a X_data line into a vector 
#beacause bert embedding has been saved as a list of strings
def vectBert(l):
  #bert embedding of dim768 and User-ID
  b = np.zeros(768+1)

  #1st copy of User-ID
  b[0] = (l[0]-MIN)/(MAX-MIN)

  #THen handling of list of strings
  v = X_data.iloc[0][1].split("'")
  b[1:] = [float(v[i]) for i in range(1, len(v), 2)]
  
  return b

In [11]:
print(len(vectBert(X_data.iloc[0])))
vectBert(X_data.iloc[0])[0:5]

769


array([ 0.99236496, -0.20497975,  0.03223878,  0.0620426 , -0.38984668])

In [12]:
#treatment of all lines of X_data such that X_data.book_description is not empty
def matBert(l, df):
  m = np.zeros((len(l), (768+1)))
  
  ind = 0
  for i in l.index:
      m[ind] = vectBert(df.iloc[i])
      ind = ind + 1
  
  return m

In [13]:
#It takes about 1min for about 75 000 lines
mat_X_data = matBert(lineOK, X_data)
mat_X_data.shape

(75976, 769)

In [14]:
#treatment of all corresponding lines of Y_data
#=> same lines as lines for which X_data.book_description is not empty
def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [15]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(75976, 1)

In [16]:
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [17]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 75976 

X_train.shape (60780, 769)
X_test.shape (15196, 769) 

y_train.shape (60780, 1)
y_test.shape (15196, 1)


In [129]:
type(X_train)

numpy.ndarray

In [130]:
X_train[:5]

array([[ 0.19897004, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.24817641, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.55151589, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.93762148, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.49270565, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486]])

In [65]:
y_train[0]

array([0])

# Ordinal regression

### Model training

In [18]:
!pip install mord
from mord.threshold_based import LogisticAT

  Created wheel for mord: filename=mord-0.6-cp37-none-any.whl size=6007 sha256=9564f1fc7d0d9b03aa356c215d58a3e3d7c3f273c3fed81585ee85f844578953
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


In [19]:
LogisticAT().get_params().keys()

dict_keys(['alpha', 'max_iter', 'verbose'])

In [21]:
np.logspace(3, 5, 5)

array([  1000.        ,   3162.27766017,  10000.        ,  31622.77660168,
       100000.        ])

In [146]:
dico_param = {
    'max_iter': np.logspace(3, 5, 5).astype(int),
    'alpha': np.logspace(0, 4, 5)
}

recherche_hyper = GridSearchCV(estimator = LogisticAT(), \
			param_grid = dico_param, scoring="accuracy", cv=5)

In [20]:
Counter(np.squeeze(y_train))

Counter({0: 36835,
         1: 106,
         2: 161,
         3: 358,
         4: 514,
         5: 2616,
         6: 2026,
         7: 4246,
         8: 5818,
         9: 3643,
         10: 4457})

In [21]:
LAT = LogisticAT(max_iter = 10000)
LAT.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticAT(alpha=1.0, max_iter=10000, verbose=0)

### Model validation

In [22]:
Counter(np.squeeze(y_test))

Counter({0: 9209,
         1: 26,
         2: 40,
         3: 90,
         4: 129,
         5: 654,
         6: 506,
         7: 1062,
         8: 1455,
         9: 911,
         10: 1114})

In [23]:
#always predicts the most represented class !
y_pred = LAT.predict(X_test)

In [24]:
y_pred.shape

(15196,)

In [25]:
Counter(y_pred)

Counter({0: 15196})

In [26]:
LAT.save("/content/drive/MyDrive/ProjetML/OrdinalRegression.model")

AttributeError: ignored